In [1]:
# Connection Strings - json

# Connection Strings - SQL database (where we are writing to)

# Translate json file into objects

# Fstrings to write dynamic SQL code into tables

# Close connection strings

In [186]:
# Import Depencies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from apikey import apikey

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [208]:
import requests
from requests.structures import CaseInsensitiveDict

#get latitude, longitude, and radius for starting point - New York City is [40.712776 N, -74.005974 W], Germany is [51.1657 N, 10.4515 E]  Germany country code is "de"
#London is [51.507351, -0.127758] Brisbane is [-27.469770, 153.025131]
#I can find London, but US cities are not coming up


default = [0, 0]
germany = [51.1657, 10.4515]
london = [51.507351, -0.127758]
brisbane = [-27.469770, 153.025131]
custom = [0,0] #use your own coordinates - probably not necessary


#set the location to search
location_search = germany


lat = "-27.469770"
lon = "153.025131"


radius_meters = 5000
country_code = ''
limit = 500
offset = 0 #default is zero

location_filter = f'circle:{location_search[1]},{location_search[0]},{radius_meters}'

if country_code != "":
    country_code = country_code + f"|countrycode:{country_code}"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


In [188]:
# List of categories
# catering, commercial.food_and_drinks, accomodations, entertainment, leisure, parking, wheelchair
category_string = ""

activity = True
commercial = True
commercial_catering = False
commercial_supermarket = True
accomodations = False
entertainment = True
leisure = False
parking = False
wheelchair = False


In [209]:
# dictionary of each category that will be captured
category_dict = {'activity' : activity, 'commercial' : commercial, 'commerical.catering' : commercial_catering, 'accomodation' : accomodations, \
                        'entertainment' : entertainment, 'leisure': leisure, 'parking' : parking, 'wheelchair.yes' : wheelchair}

string_of_categories = ""

for k,v in category_dict.items():
    if v:
        string_of_categories = string_of_categories + k + ','

#remove last comma
string_of_categories = string_of_categories[:-1]

# conditions, organic.only, vegetarian.only, wheelchair.yes
url2 = f"https://api.geoapify.com/v2/places?categories={string_of_categories}&filter={location_filter}&limit={limit}&offset={offset}&apiKey={apikey}"

# atms in radius, 

# contact information for the restaurants, opening/closing hours, web address, phone number

In [210]:
# get API dictionary
resp = requests.get(url2, headers=headers)

print(resp.status_code)

json = resp.json()
json

200


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Thüringentherme',
    'country': 'Germany',
    'country_code': 'de',
    'state': 'Thuringia',
    'county': 'Unstrut-Hainich-Kreis',
    'city': 'Mühlhausen',
    'postcode': '99974',
    'street': 'Lindenbühl',
    'housenumber': '10',
    'lon': 10.45631153134633,
    'lat': 51.2052761,
    'formatted': 'Thüringentherme, Lindenbühl 10, 99974 Mühlhausen, Germany',
    'address_line1': 'Thüringentherme',
    'address_line2': 'Lindenbühl 10, 99974 Mühlhausen, Germany',
    'categories': ['building', 'entertainment', 'entertainment.water_park'],
    'details': ['details', 'details.contact', 'details.facilities'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Thüringentherme',
      'sauna': 'yes',
      'osm_id': 98992743,
  